In [1]:
import requests #use for handle requests
import time #use for set sleep time
from bs4 import BeautifulSoup #use for parse html files
import string #use for punctuation
import json #use for write & read json files
import pandas as pd #use for create dataframe
import nltk
from nltk.corpus import stopwords #use for detect stopwords in string
from nltk.tokenize import word_tokenize #use for getting words in string
from nltk.stem import PorterStemmer #use for stemmer of the word
from langdetect import detect

In [ ]:
book_url_list = []
url_format = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="
for i in range(300):
    url = url_format + str(i+1)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    first_soup = soup.find_all("div", class_="js-tooltipTrigger tooltipTrigger")
    for item in first_soup:
        second_soup = BeautifulSoup(str(item), "html.parser")
        book_link_tag = str(second_soup.find_all("a"))
        first_split = book_link_tag.split(">")
        book_just_links = first_split[0]
        second_split = book_just_links.split("href=")
        book_url_with_title = second_split[1]
        third_split = book_url_with_title.split(" title")
        book_url_with_qoutation = third_split[0]
        book_url = "https://www.goodreads.com" + book_url_with_qoutation.strip('"')
        book_url_list.append(book_url)
with open("book url file.txt", "w") as book_url_file:
    json.dump(book_url_list, book_url_file)

In [ ]:
with open("book url file.txt", "r") as book_url_file:
    book_urls = json.load(book_url_file)
html_file_path_format = "/Users/vahidmohammadi/™️/DS/Practice/project/07/HTML Files/page "
for counter_html in range(28020,len(book_urls)):
    book = book_urls[counter_html]
    try:
        response = requests.get(book)
        text = response.text
        html_file_path = html_file_path_format + str(counter_html + 1) + ".html"
        with open(html_file_path, "w") as book_html_file:
            book_html_file.write(text)
        time.sleep(5)
    except:
        continue

In [2]:
with open("book url file.txt", "r") as book_url_file:
    book_urls = json.load(book_url_file)
html_file_path_format = "/Users/vahidmohammadi/™️/DS/Practice/project/07/HTML Files/page "
tsv_file_path_format = "/Users/vahidmohammadi/™️/DS/Practice/project/07/TSV File/page "
for counter_tsv in range(len(book_urls)):
    try:
        html_file_path = html_file_path_format + str(counter_tsv + 1) + ".html"
        with open(html_file_path, "r") as book_html_file:
            content = book_html_file.read()
        soup = BeautifulSoup(content, "html.parser")
        try:
            title = soup.find_all("h1", id="bookTitle")[0].text.strip("\n").strip()
        except:
            title = ""
        try:
            author = soup.find_all("a", class_="authorName")[0].text.strip("\n").strip()
        except:
            author = ""
        try:
            rating_star = soup.find_all("span", itemprop="ratingValue")[0].text.strip("\n").strip()
        except:
            rating_star = ""
        try:
            rating_counts = soup.find_all("meta", itemprop="ratingCount")[0].get("content", None)
        except:
            rating_counts = ""
        try:
            review_counts = soup.find_all("meta", itemprop="reviewCount")[0].get("content", None)
        except:
            review_counts = ""
        try:
            plot = soup.find_all("div", id="description")[0].text.strip("\n").replace("\n","")
            if detect(plot) != "en":
                continue
        except:
            plot = ""
        try:
            number_of_pages = soup.find_all("div", id="details")[0].find_all("span", itemprop="numberOfPages")[0].text
        except:
            number_of_pages = ""
        try:
            date = soup.find_all("div", id="details")[0].find_all("div", class_="row")[1].text.strip().split("\n")[1].strip().strip("\n")
        except:
            date = ""
        try:
            info_box = soup.find_all("div", id="details")[0].find_all("div", class_=["infoBoxRowTitle", "infoBoxRowItem"])
            for i in range(len(info_box)):
                try:
                    if info_box[i].text == "Characters":
                        characters = info_box[i+1].text.strip("\n")
                except:
                    characters = ""
                try:
                    if info_box[i].text == "Setting":
                        setting = info_box[i+1].text.replace("\n", " ")
                except:
                    setting = ""
                try:
                    if info_box[i].text == "Series":
                        series = info_box[i+1].text
                except:
                    series = ""
        except:
            characters = ""
            setting = ""
            series = ""
        tsv_file_format = title+"\t"+series+"\t"+author+"\t"+rating_star+"\t"+rating_counts+"\t"+review_counts\
        +"\t"+plot+"\t"+number_of_pages+"\t"+date+"\t"+characters+"\t"+setting
        tsv_file_path = tsv_file_path_format + str(counter_tsv + 1) + ".tsv"
        with open(tsv_file_path, "w") as book_tsv_file:
            book_tsv_file.write(tsv_file_format)
    except:
        continue

In [9]:
#this part is used for removing stopwords punctuation and get the stem words in tsv files and
#indexing each new words and create dictionary of words exist on movie's tsv file
with open("book url file.txt", "r") as book_url_file:
    book_urls = json.load(book_url_file)
words = {} #for indexing each word
books_words = {} #for existance of each word in movie's tsv file
word_id = 0
counter_nltk = 0
exclude = set(string.punctuation)
stop_words = set(stopwords.words('english'))
ps = PorterStemmer() #use for finding stem of each word

for counter_nltk in range(len(book_urls)):
    try:
        file_name_tsv = "/Users/vahidmohammadi/™️/DS/Practice/project/07/TSV File/page " + str(counter_nltk+1)+".tsv"#tsv file path
        with open(file_name_tsv, "r") as book_tsv_file:#open tsv files
            text_list = book_tsv_file.read()
            text = " ".join(text_list.split("\t")[1:3])
            text = ''.join(ch for ch in text if ch not in exclude) #use for getting each word without white spaces
            word_tokens = word_tokenize(text)
            filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]#removing stopwords and finding stem of the each word
            for word in filtered_sentence:
                if word not in words.values():#looking for NOT existance of word in words list for indexing
                    words[word_id] = word
                    books_words[word_id] = [counter_nltk+1]#adding movies name in movies_words
                    word_id += 1
                else: # if word exist in words list just add movies ID (movies file name that produce by counter) in movies word
                    exist_id = [key for key, value in words.items() if value == word][0]
                    if counter_nltk+1 not in books_words[exist_id]:
                        books_words[exist_id].append(counter_nltk+1)
    except:
        continue
with open("words_index.json", "w") as word_index: #write words list in json file
    json.dump(words, word_index)
with open("books_words.json", "w") as books_words_file:#write movies_words list in json file
    json.dump(books_words, books_words_file)

In [19]:
#this part is used for searching through json files and find first 5 best match and show them in pandas frame work
print("================================")
print("|Which book would you recomend?|")
print("================================")
print("--------------------")
print("|Search Engine v1.0|")
print("--------------------")
our_input = input("search: ")
exclude = set(string.punctuation)
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
our_input = ''.join(ch for ch in our_input if ch not in exclude)
word_tokens = word_tokenize(our_input)
filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]
words_in_input = []
with open("words_index.json", "r") as word_index:
    words = json.load(word_index)
with open("books_words.json", "r") as books_words_file:
    books_words = json.load(books_words_file)
for word in filtered_sentence:
    for key, value in words.items():
        if word == value:
            words_in_input.append(key)
words_in_books = []
for key_of_input in words_in_input:#finding each searched word's index
    for key_of_books, value in books_words.items():
        if key_of_input == key_of_books:
            words_in_books.append(value)
final_result_dict = {}#use for saving matches
for first_item in words_in_books:#searching by words index that exist in movies words list
    for second_item in first_item:
        if second_item in final_result_dict.keys():
            final_result_dict[second_item] += 1
        else:
            final_result_dict[second_item] = 1
all_item_match_dict = {}
for key, value in final_result_dict.items():
    if value == len(words_in_input):
        all_item_match_dict[key] = value
print("")
print("Result: ")
df = {"book":[], "plot":[], "link":[]}#use for creating pandas dataframe
with open("book url file.txt", "r") as book_url_file:
    books_urls = json.load(book_url_file)
for item in all_item_match_dict:
    file_name = "/Users/vahidmohammadi/™️/DS/Practice/project/07/TSV File/page " + str(item) + ".tsv"#tsv file path
    with open(file_name, "r") as target:#open tsv file for reading title and intro
        tsv_file = target.read()
        book_list = tsv_file.split("\t")
        df["book"].append(book_list[0])
        df["plot"].append(book_list[6])
        df["link"].append(books_urls[item-1])
pd_df = pd.DataFrame(df)
if pd_df.empty:
    print("NOT Found 404")
pd_df.head(5)

|Which book would you recomend?|
--------------------
|Search Engine v1.0|
--------------------
search: love story

Result: 


,book,plot,link
0,Love Story,"Oliver Barrett IV, a wealthy jock from a stuff...",https://www.goodreads.com/book/show/73968.Love...
1,Bloodsucking Fiends,There is an alternate cover edition here.Jody ...,https://www.goodreads.com/book/show/33454.Bloo...
